In [5]:
from google.cloud import bigquery
import os

# Set your Google Cloud credentials
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/davide.zanutto/Library/CloudStorage/OneDrive-IKEA/Desktop/Keys/ingka-feed-student-dev-a586db13efed.json"

# Initialize a BigQuery client
client = bigquery.Client()

# Define your query
query = """ SELECT art_id, concat(title,'. ',text) as text FROM `ingka-feed-student-dev.RR.RatingsReviews` AS rr
            INNER JOIN `ingka-feed-student-dev.RR.product_categories` AS pc 
            ON rr.art_id = SPLIT(pc.global_id, ',')[SAFE_OFFSET(1)] 
            WHERE country_code = 'us' and PRODUCT_AREA = 'Open storage'
            ORDER BY inserted_on DESC """

# Run the query and export to CSV
query_job = client.query(query)
df = query_job.to_dataframe()
df['text'] = df['text'].replace('\n', ' ', regex=True)

df.to_csv('artId_reviews.csv', index=False)

In [7]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('artId_reviews.csv')

# Loop through the dataframe grouped by 'article_id'
for article_id, group in df.groupby('art_id'):
    # Define the path for the output CSV file
    output_path = f'articles/{article_id}.csv'
    # Save each group to a separate CSV file
    group.to_csv(output_path, index=False)

In [6]:
import pandas as pd
import os

def quote_keyphrases(file_path):
    try:
        # Attempt to read the CSV normally
        df = pd.read_csv(file_path)
        # Apply quotes
        df['Keyphrases'] = df['Keyphrases'].apply(lambda x: f'"{x}"')
        df.to_csv(file_path, index=False)
    except pd.errors.ParserError:
        # Handle files with inconsistent column issues
        with open(file_path, 'r') as file:
            lines = file.readlines()
        
        # Assuming 'Keyphrases' is always the second column
        with open(file_path, 'w') as file:
            for line in lines:
                if not line.strip():
                    continue
                parts = line.split(',')
                keyphrase = ','.join(parts[1:])  # Join all parts except the first column
                keyphrase = f'"{keyphrase.strip()}"'  # Strip and quote
                file.write(f"{parts[0]},{keyphrase}\n")

def process_folder(folder_path):
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if file_path.endswith('.csv'):
            quote_keyphrases(file_path)
            print(f'Processed {filename}')

folder_path = '../articles/processed'
process_folder(folder_path)


Processed 9275085.csv


KeyError: 'Keyphrases'